In [17]:
def getLetter():
    input_letter = input()
    if input_letter.isalpha() and len(input_letter) == 1:
        return input_letter.lower()
    else:
        print('Error, try repeat')
        return getLetter()

In [18]:
class Letter():
    def __init__(self, letter, open_flag):
        self.letter = letter
        self.open_flag = open_flag
    
    def show(self):
        if self.open_flag:
            return self.letter
        else:
            return '*'

In [24]:
class Slovo():
    def __init__(self, word, attemts_number):
        self.word = word
        self.attemts_number = attemts_number

        letter_list = []
        for i in word:
            letter_list.append(Letter(i, False))
        self.letter_list = letter_list

        inputs_list = []
        self.inputs_list = inputs_list

    def show(self, show_word_flag, input_letter=None):
        new_word = ''
        if show_word_flag:
            return self.word
        else:
            is_letter_in_word = False
            if input_letter not in self.inputs_list:
                self.inputs_list.append(input_letter)
            else:
                print('You enter this letter yet')
                is_letter_in_word = True
            for i in self.letter_list:
                if i.letter == input_letter:
                    is_letter_in_word = True
                    i.open_flag = True
                new_word += i.show()
            if not is_letter_in_word:
                 print(f'Letter "{input_letter}" not in word')
                 self.attemts_number -=1
                
            return (new_word, self.attemts_number)
            


In [25]:
def game(word):
    num_attemts = 7
    slovo = Slovo(word, num_attemts)
    while num_attemts != 0:
        input_letter = getLetter()
        new_word, num_attemts = slovo.show(False, input_letter)
        print(f'{new_word} \nAttempts left: {num_attemts}')

        if new_word == slovo.show(True):
            print('You win')
            break
    if num_attemts == 0:  
        print('Game over')

In [27]:
game('language')

KeyboardInterrupt: Interrupted by user